# LLM Message History

Large Language Models are inherently stateless and have no knowledge of previous interactions with a user, or even of previous parts of the current conversation. While this may not be noticeable when asking simple questions, it becomes a hindrance when engaging in long running conversations that rely on conversational context.

The solution to this problem is to append the previous conversation history to each subsequent call to the LLM.

This notebook will show how to use Redis to structure and store and retrieve this conversational message history.

In [1]:
// Load Maven dependencies
%maven redis.clients:jedis:6.2.0
%maven org.slf4j:slf4j-nop:2.0.16
%maven com.fasterxml.jackson.core:jackson-databind:2.18.0
%maven com.fasterxml.jackson.dataformat:jackson-dataformat-yaml:2.18.0
%maven com.github.f4b6a3:ulid-creator:5.2.3

// Import RedisVL classes
import com.redis.vl.extensions.messagehistory.MessageHistory;

// Import Redis client
import redis.clients.jedis.UnifiedJedis;
import redis.clients.jedis.HostAndPort;

// Import Java standard libraries
import java.util.*;

In [2]:
// Connect to Redis
UnifiedJedis client = new UnifiedJedis(new HostAndPort("redis-stack", 6379));

SLF4J: Failed to load class "org.slf4j.impl.StaticLoggerBinder".
SLF4J: Defaulting to no-operation (NOP) logger implementation
SLF4J: See http://www.slf4j.org/codes.html#StaticLoggerBinder for further details.


In [3]:
// Create a MessageHistory instance
MessageHistory chatHistory = new MessageHistory("student tutor", client);

## Storing Messages

To align with common LLM APIs, Redis stores messages with `role` and `content` fields.
The supported roles are "system", "user" and "llm".

You can store messages one at a time or all at once.

In [4]:
// Add a system message
chatHistory.addMessage(
    Map.of(
        "role", "system",
        "content", "You are a helpful geography tutor, giving simple and short answers to questions about European countries."
    )
);

// Add multiple messages at once
chatHistory.addMessages(
    List.of(
        Map.of("role", "user", "content", "What is the capital of France?"),
        Map.of("role", "llm", "content", "The capital is Paris."),
        Map.of("role", "user", "content", "And what is the capital of Spain?"),
        Map.of("role", "llm", "content", "The capital is Madrid."),
        Map.of("role", "user", "content", "What is the population of Great Britain?"),
        Map.of("role", "llm", "content", "As of 2023 the population of Great Britain is approximately 67 million people.")
    )
);

## Retrieving Conversation History

At any point we can retrieve the recent history of the conversation. It will be ordered by entry time.

In [5]:
// Get recent messages (default top_k=5)
List<Map<String, Object>> context = chatHistory.getRecent(5, false, false, null);

for (Map<String, Object> message : context) {
    System.out.println(message);
}

{role=llm, content=The capital is Paris.}
{role=user, content=And what is the capital of Spain?}
{role=llm, content=The capital is Madrid.}
{role=user, content=What is the population of Great Britain?}
{role=llm, content=As of 2023 the population of Great Britain is approximately 67 million people.}


## Storing Prompt/Response Pairs

In many LLM flows the conversation progresses in a series of prompt and response pairs. Message history offers a convenience function `store()` to add these simply.

In [6]:
String prompt = "what is the size of England compared to Portugal?";
String response = "England is larger in land area than Portugal by about 15000 square miles.";
chatHistory.store(prompt, response);

// Get recent 6 messages
List<Map<String, Object>> updatedContext = chatHistory.getRecent(6, false, false, null);

for (Map<String, Object> message : updatedContext) {
    System.out.println(message);
}

{role=user, content=And what is the capital of Spain?}
{role=llm, content=The capital is Madrid.}
{role=user, content=What is the population of Great Britain?}
{role=llm, content=As of 2023 the population of Great Britain is approximately 67 million people.}
{role=user, content=what is the size of England compared to Portugal?}
{role=llm, content=England is larger in land area than Portugal by about 15000 square miles.}


## Managing Multiple Users and Conversations

For applications that need to handle multiple conversations concurrently, Redis supports tagging messages to keep conversations separated.

In [7]:
// Add a system message for a different session
chatHistory.addMessage(
    Map.of(
        "role", "system",
        "content", "You are a helpful algebra tutor, giving simple answers to math problems."
    ),
    "student two"
);

// Add math-related messages to "student two" session
chatHistory.addMessages(
    List.of(
        Map.of("role", "user", "content", "What is the value of x in the equation 2x + 3 = 7?"),
        Map.of("role", "llm", "content", "The value of x is 2."),
        Map.of("role", "user", "content", "What is the value of y in the equation 3y - 5 = 7?"),
        Map.of("role", "llm", "content", "The value of y is 4.")
    ),
    "student two"
);

// Retrieve messages for "student two" session
List<Map<String, Object>> mathMessages = chatHistory.getRecent(10, false, false, "student two");

System.out.println("Math messages for 'student two':");
for (Map<String, Object> message : mathMessages) {
    System.out.println(message);
}

Math messages for 'student two':
{role=system, content=You are a helpful algebra tutor, giving simple answers to math problems.}
{role=user, content=What is the value of x in the equation 2x + 3 = 7?}
{role=llm, content=The value of x is 2.}
{role=user, content=What is the value of y in the equation 3y - 5 = 7?}
{role=llm, content=The value of y is 4.}


## Retrieving Messages as Text

You can also retrieve messages as simple text strings (just the content) instead of full message objects.

In [8]:
// Get messages as text (asText=true)
List<String> textMessages = chatHistory.getRecent(5, true, false, null);

System.out.println("Messages as text:");
for (String message : textMessages) {
    System.out.println(message);
}

Messages as text:
The capital is Madrid.
What is the population of Great Britain?
As of 2023 the population of Great Britain is approximately 67 million people.
what is the size of England compared to Portugal?
England is larger in land area than Portugal by about 15000 square miles.


## Getting All Messages

You can retrieve all messages for the default session using the `getMessages()` method.

In [9]:
// Get all messages for the default session
List<Map<String, Object>> allMessages = chatHistory.getMessages();

System.out.println("All messages (" + allMessages.size() + " total):");
for (Map<String, Object> message : allMessages) {
    System.out.println(message);
}

All messages (9 total):
{role=system, content=You are a helpful geography tutor, giving simple and short answers to questions about European countries.}
{role=user, content=What is the capital of France?}
{role=llm, content=The capital is Paris.}
{role=user, content=And what is the capital of Spain?}
{role=llm, content=The capital is Madrid.}
{role=user, content=What is the population of Great Britain?}
{role=llm, content=As of 2023 the population of Great Britain is approximately 67 million people.}
{role=user, content=what is the size of England compared to Portugal?}
{role=llm, content=England is larger in land area than Portugal by about 15000 square miles.}


## Conversation Control

LLMs can hallucinate on occasion and when this happens it can be useful to prune incorrect information from conversational histories so this incorrect information doesn't continue to be passed as context.

In [10]:
// Store an incorrect response
chatHistory.store(
    "what is the smallest country in Europe?",
    "Monaco is the smallest country in Europe at 0.78 square miles." // Incorrect - Vatican City is smallest
);

// Get the key of the incorrect message (raw=true returns all fields including entry_id)
List<Map<String, Object>> rawContext = chatHistory.getRecent(1, false, true, null);
String badKey = (String) rawContext.get(0).get("entry_id");

System.out.println("Bad message key: " + badKey);

// Drop the incorrect response
chatHistory.drop(badKey);

// Verify the incorrect response is gone
List<Map<String, Object>> correctedContext = chatHistory.getRecent(5, false, false, null);

System.out.println("\nCorrected context:");
for (Map<String, Object> message : correctedContext) {
    System.out.println(message);
}

Bad message key: 01K713B8XBCP1V3XX3M625YTG7:1.7598999199724438E9

Corrected context:
{role=user, content=What is the population of Great Britain?}
{role=llm, content=As of 2023 the population of Great Britain is approximately 67 million people.}
{role=user, content=what is the size of England compared to Portugal?}
{role=llm, content=England is larger in land area than Portugal by about 15000 square miles.}
{role=user, content=what is the smallest country in Europe?}


## Cleanup

Clear the conversation history when done.

In [11]:
// Clear all messages (but keep the index)
chatHistory.clear();
System.out.println("Cleared all messages");

// Or delete everything including the index
chatHistory.delete();
System.out.println("Deleted message history index");

// Close the Redis connection
client.close();
System.out.println("Redis connection closed");

Cleared all messages
Deleted message history index
Redis connection closed
